In [3]:
import sys, os, time, shutil, random, pickle
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
import omnifig as fig
import h5py as hf
import numpy as np
np.set_printoptions(linewidth=120)
from tabulate import tabulate
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
from torch import optim as opt
from torch.utils.data import Dataset, DataLoader, TensorDataset

%matplotlib notebook
import matplotlib.pyplot as plt

import trifinger_simulation
from trifinger_simulation import trifinger_platform, sample

fig.initialize()

import omnilearn as learn
from omnilearn import models
from omnilearn import util
from omnilearn.data import InterventionSamplerBase
from omnilearn import viz as viz_util

In [4]:
platform = trifinger_simulation.TriFingerPlatform(visualization=True)


In [5]:

# Move the fingers to random positions
while True:
    position = platform.spaces.robot_position.gym.sample()
    finger_action = platform.Action(position=position)

    # apply action for a few steps, so the fingers can move to the target
    # position and stay there for a while
    for _ in range(100):
        t = platform.append_desired_action(finger_action)
        # sleep after each step so that the visualization happens in real
        # time
        time.sleep(platform.get_time_step())

    # show the latest observations
    robot_observation = platform.get_robot_observation(t)
    print("Finger0 Joint Positions: %s" % robot_observation.position[:3])

    # the cube pose is part of the camera observation
    camera_observation = platform.get_camera_observation(t)
    cube_pose = camera_observation.object_pose
    print("Cube Position (x, y, z): %s" % cube_pose.position)

Finger0 Joint Positions: [ 0.1243733   0.64421564 -1.11594181]
Cube Position (x, y, z): [0.     0.     0.0325]
Finger0 Joint Positions: [ 0.12437352  0.64421534 -1.11594122]
Cube Position (x, y, z): [-4.27172987e-18 -5.26066929e-19  3.34897620e-02]
Finger0 Joint Positions: [ 0.42489821  0.81377157 -1.50300435]
Cube Position (x, y, z): [ 0.0416344  -0.00111306  0.03608894]
Finger0 Joint Positions: [ 0.42702009  0.55897947 -1.9701765 ]
Cube Position (x, y, z): [0.05106157 0.01325824 0.03460562]
Finger0 Joint Positions: [ 0.59038254  0.29310811 -2.02514002]
Cube Position (x, y, z): [0.05650392 0.01982474 0.03575518]


KeyboardInterrupt: 